<a href="https://colab.research.google.com/github/sko9370/rootCauseAnalysis/blob/main/Individual_Logs_Starter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# download and uncompress the logs
!wget https://github.com/sko9370/rootCauseAnalysis/raw/main/logs.zip
!unzip logs.zip
!rm logs.zip

--2021-05-03 12:50:29--  https://github.com/sko9370/rootCauseAnalysis/raw/main/logs.zip
Resolving github.com (github.com)... 140.82.114.4
Connecting to github.com (github.com)|140.82.114.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/sko9370/rootCauseAnalysis/main/logs.zip [following]
--2021-05-03 12:50:29--  https://raw.githubusercontent.com/sko9370/rootCauseAnalysis/main/logs.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 12896935 (12M) [application/zip]
Saving to: ‘logs.zip’

logs.zip            100%[===================>]  12.30M  --.-KB/s    in 0.07s   

2021-05-03 12:50:29 (174 MB/s) - ‘logs.zip’ saved [12896935/12896935]

Archive:  logs.zip
   creating: logs/
  inflating: logs/bit

In [2]:
# install package to be able to load in json logs
!pip install pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()
spark.conf.set("spark.sql.caseSensitive", "true")

     |████████████████████████████████| 212.3MB 73kB/s 
     |████████████████████████████████| 204kB 48.5MB/s 
  Created wheel for pyspark: filename=pyspark-3.1.1-py2.py3-none-any.whl size=212767604 sha256=ba9ebff741ca21017e008d46f1867cb86aeff335fb1d313515788d54ef626ddf
  Stored in directory: /root/.cache/pip/wheels/0b/90/c0/01de724414ef122bd05f056541fb6a0ecf47c7ca655f8b3c0f
Successfully built pyspark


In [3]:
# ingest firewall.json log
firewall_df = spark.read.json("logs/firewall.json")
firewall_df.createTempView("firewall")

# ingest sysmon.json log
sysmon_df = spark.read.json("logs/sysmon.json")
sysmon_df.createTempView("sysmon")

# ingest remoteConnection.json log
remoteConnection_df = spark.read.json("logs/remoteConnection.json")
remoteConnection_df.createTempView("remoteConnection")

In [4]:
# run a sql query on previously ingested "firewall" log. use the name from the .createTempView method
# DESCRIBE is a sql function that lists all the columns and the type of data they use
firewall = spark.sql(
'''
DESCRIBE firewall
'''
)
# .show(<number of rows to show>, <truncate means to cut off columns to make them fit on the screen)
firewall.show(100, truncate = False)

+--------------------+-------------+-------+
|col_name            |data_type    |comment|
+--------------------+-------------+-------+
|@timestamp          |string       |null   |
|@version            |string       |null   |
|AccountName         |string       |null   |
|AccountType         |string       |null   |
|Action              |string       |null   |
|Active              |string       |null   |
|Channel             |string       |null   |
|Direction           |string       |null   |
|Domain              |string       |null   |
|EdgeTraversal       |string       |null   |
|EmbeddedContext     |string       |null   |
|EventID             |bigint       |null   |
|EventReceivedTime   |string       |null   |
|EventTime           |string       |null   |
|EventType           |string       |null   |
|ExecutionProcessID  |bigint       |null   |
|Flags               |string       |null   |
|Hostname            |string       |null   |
|InterfaceGuid       |string       |null   |
|Interface

In [5]:
# same thing with sysmon here
sysmon = spark.sql(
'''
DESCRIBE sysmon
'''
)
# new option here, vertical lists the columns vertically instead of horizontally like the above
# this is useful for fitting log entries with MANY columns on one screen
sysmon.show(100, truncate = False, vertical = True)

-RECORD 0----------------------------
 col_name  | @timestamp              
 data_type | string                  
 comment   | null                    
-RECORD 1----------------------------
 col_name  | @version                
 data_type | string                  
 comment   | null                    
-RECORD 2----------------------------
 col_name  | AccountName             
 data_type | string                  
 comment   | null                    
-RECORD 3----------------------------
 col_name  | AccountType             
 data_type | string                  
 comment   | null                    
-RECORD 4----------------------------
 col_name  | Archived                
 data_type | string                  
 comment   | null                    
-RECORD 5----------------------------
 col_name  | CallTrace               
 data_type | string                  
 comment   | null                    
-RECORD 6----------------------------
 col_name  | Channel                 
 data_type |

In [ ]:
remoteConnection = spark.sql(
'''
DESCRIBE remoteConnection
'''
)
remoteConnection.show(100, truncate = False)

In [6]:
# a more complex query that counts the number of each EventID in the sysmon log and orders them in descending (DESC) order
sysmon = spark.sql(
'''
SELECT EventID, COUNT(EventID) AS MOST_FREQUENT
FROM sysmon
GROUP BY EventID
ORDER BY COUNT(EventID) DESC
'''
)
sysmon.show(100, truncate = False)

+-------+-------------+
|EventID|MOST_FREQUENT|
+-------+-------------+
|12     |61151        |
|10     |39283        |
|7      |20259        |
|13     |17541        |
|11     |1649         |
|3      |1229         |
|9      |652          |
|1      |447          |
|23     |422          |
|5      |401          |
|18     |362          |
|2      |209          |
|8      |95           |
|17     |84           |
|22     |81           |
|15     |18           |
|4      |1            |
+-------+-------------+



In [9]:
# iterates through each column and counts the occurance of each unique entry and orders in descending order
# notice this does not use the temporary view (sql) and instead references the data directly through the dataframe (sysmon_df)
most_frequent = []
for col in sysmon_df.columns:
    most_frequent.append(sysmon_df.groupBy(col).count().orderBy('count',ascending = False))

In [11]:
# this actually prints out the top 5 most frequent entries for each column starting with the 5th column (previous ones were mostly unique or uninteresting)
# this will help you determine which columns are useful to actually query on or investigate
for col in most_frequent[5:]:
    col.show(5, truncate = False)

+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+------+
|CallTrace                                                                   

KeyboardInterrupt: ignored

In [12]:
# yet another method to query using a more code-like syntax if you are unfamiliar with sql. very similar keywords however
# also note that this method uses the dataframe (sysmon_df) directly instead of the temporary view (sql)
sysmonEid1 = sysmon_df.filter("EventID == 1").groupBy('Image').count().orderBy('count',ascending = False).show(30,truncate = False)

+------------------------------------------------------------------------------------------------------------------------------------+-----+
|Image                                                                                                                               |count|
+------------------------------------------------------------------------------------------------------------------------------------+-----+
|C:\Windows\System32\svchost.exe                                                                                                     |100  |
|C:\Windows\System32\conhost.exe                                                                                                     |65   |
|C:\Windows\System32\backgroundTaskHost.exe                                                                                          |18   |
|C:\Windows\System32\wbem\WmiPrvSE.exe                                                                                               |17   |
|C:\Windows\S